<a href="https://colab.research.google.com/github/fayexu/IDL_PJ_21Spring/blob/master/generate_both.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 17.0MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
     |████████████████████████████████| 901kB 52.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, RobertaTokenizer, RobertaForMaskedLM

# load tokenizer and pretrained model(Bert)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForMaskedLM.from_pretrained('roberta-base')
model.eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
def predictMask_single(text, masked_index, k):

  tokenized_text = tokenizer.tokenize(text)
  

  # mask the text
  tokenized_text[masked_index] = '[MASK]'
  tokenized_text.insert(0,'[CLS]')
  tokenized_text.append('[SEP]')
  print("masked text: ")
  print(tokenized_text)

  # tokenizie
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  encoded_dict = tokenizer(text)

  # convert to tensor
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor(encoded_dict['token_type_ids'])

  # put on cuda
  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')
  model.to('cuda')

  # predict
  with torch.no_grad():
      outputs = model(tokens_tensor, token_type_ids=segments_tensors)
      predictions = outputs[0]

  # decode
  #predicted_index = torch.argmax(predictions[0, masked_index]).item()
  predicted_indexes = torch.topk(predictions[0, masked_index + 1], k)
  top_k_tokens = zip(predicted_indexes.indices.tolist(), predicted_indexes.values.tolist())
  for token, score in top_k_tokens:
    print(tokenizer.convert_ids_to_tokens([token])[0], f"(score: {score})")

In [ ]:
def predict_single_without_id(text, k):

  tokenized_text = tokenizer.tokenize(text)

  masked_index = tokenized_text.index('[MASK]')

  # mask the text
  tokenized_text[masked_index] = '[MASK]'
  tokenized_text.insert(0,'[CLS]')
  tokenized_text.append('[SEP]')
  #print("masked text: ")
  #print(tokenized_text)

  # tokenizie
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  encoded_dict = tokenizer(text)

  # convert to tensor
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor(encoded_dict['token_type_ids'])

  # put on cuda
  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')
  model.to('cuda')

  # predict
  with torch.no_grad():
      outputs = model(tokens_tensor, token_type_ids=segments_tensors)
      predictions = outputs[0]

  # decode
  #predicted_index = torch.argmax(predictions[0, masked_index]).item()
  predicted_indexes = torch.topk(predictions[0, masked_index + 1], k)
  top_k_tokens = zip(predicted_indexes.indices.tolist(), predicted_indexes.values.tolist())
  top_k = []
  for token, score in top_k_tokens:
    top_k.append((tokenizer.convert_ids_to_tokens([token])[0], round(score, 3)))
    #print(tokenizer.convert_ids_to_tokens([token])[0], f"(score: {score})")
  return top_k

In [ ]:
line = "The belly dancer dances on stage shaking with [MASK] hips and body."
print(predict_single_without_id(line, 5))

[('her', 14.233), ('his', 13.188), ('the', 12.566), ('its', 11.525), ('both', 11.188)]


In [ ]:
male_pronoun = ['he', 'his', 'him', 'himself']
female_pronoun = ['she', 'her', 'hers', 'herself']
pronoun = male_pronoun+female_pronoun
file = open('/content/drive/My Drive/Colab Notebooks/train_full_filtered.txt', 'r')
lines = file.readlines()
# writing to file
out_file = open('out.txt', 'w')
topfive_file = open('score.txt', 'w')
for line in lines:
  topfive = predict_single_without_id(line,5)
  topfivestr = [tuples[0] for tuples in topfive]
  for i in range(4):
    if male_pronoun[i] in topfivestr or female_pronoun[i] in topfivestr:
      male_verison = line.replace("[MASK]", male_pronoun[i])
      female_verison = line.replace("[MASK]", female_pronoun[i])
      L = ["text:"+male_verison, "text:"+female_verison]
      out_file.writelines(L)
      topfive_file.write(str(topfive)+'\n')
out_file.close()
topfive_file.close()


